In [ ]:
!pip install --upgrade tensorflow

In [ ]:
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

In [ ]:
from keras.datasets import cifar100
from sklearn.model_selection import train_test_split
from tensorflow.keras import utils as np_utils
from keras.utils import to_categorical

# load the CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

# split the training dataset into sub-training set and validation set
x_train_sub, x_val, y_train_sub, y_val = train_test_split(x_train, y_train, test_size=0.15, random_state=42)


In [ ]:
y_train_sub = to_categorical(y_train_sub, num_classes=100)
y_val = to_categorical(y_val, num_classes=100)

In [ ]:
num_classes = 100

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adam,SGD

def create_model(activation, optimizer, filters, kernel_size, pool_size, dropout_rate):
    
    model = Sequential()
    model.add(Conv2D(filters, kernel_size=kernel_size, activation=activation, padding='same', input_shape=x_train_sub.shape[1:]))
    model.add(Conv2D(filters, kernel_size=kernel_size, activation=activation, padding='same'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(dropout_rate))
    model.add(Conv2D(filters*2, kernel_size=kernel_size, activation=activation, padding='same'))
    model.add(Conv2D(filters*2, kernel_size=kernel_size, activation=activation, padding='same'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(dropout_rate))
    model.add(Flatten())
    model.add(Dense(512, activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax'))

    
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model




In [ ]:
activations = ['relu', 'tanh']
optimizers = [Adam(learning_rate=1e-3), Adam(learning_rate=1e-4),SGD(learning_rate=0.001)]
filters_list = [64,128,256]
kernel_sizes = [(3, 3), (5, 5)]
pool_sizes = [(2, 2), (3, 3)]
dropout_rates = [0.25,0.3, 0.5]

In [ ]:
models = []
for activation in activations:
    for optimizer in optimizers:
        for filters in filters_list:
            for kernel_size in kernel_sizes:
                for pool_size in pool_sizes:
                    for dropout_rate in dropout_rates:
                        with tpu_strategy.scope():# creating the model in the TPUStrategy scope means we will train the model on the TPU
                          model = create_model(activation, optimizer, filters, kernel_size, pool_size, dropout_rate)
                          history = model.fit(x_train_sub, y_train_sub, batch_size=32, epochs=20, validation_data=(x_val, y_val),verbose=1)
                          val_loss, val_acc = model.evaluate(x_val, y_val, verbose=0)
                          models.append({
                          'activation': activation,
                          'optimizer': optimizer.get_config()['name'],
                          'filters': filters,
                          'kernel_size': kernel_size,
                          'pool_size': pool_size,
                          'dropout_rate': dropout_rate,
                          'val_loss': val_loss,
                          'val_acc': val_acc,
                          'params': model.count_params(),
                          'history': history
                      })

In [ ]:
models.sort(key=lambda x: x['val_acc'], reverse=True)


top_models = models[:3]
for i, model in enumerate(top_models):
    print(f'Top {i+1} Model:')
    print('Activation:', model['activation'])
    print('Optimizer:', model['optimizer'])
    print('Filters:', model['filters'])
    print('Kernel Size:', model['kernel_size'])
    print('Pool Size:', model['pool_size'])
    print('Dropout Rate:', model['dropout_rate'])
    print('Validation Accuracy:', model['val_acc'])
    print('Parameters:', model['params'])
    print('-'*50)

In [ ]:
from keras.utils import to_categorical


y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)


for i, model in enumerate(top_models):
    print(f'Training Top {i+1} Model...')
    model = create_model(model['activation'], model['optimizer'], model['filters'], model['kernel_size'], model['pool_size'], model['dropout_rate'])
    history = model.fit(x_train, y_train, batch_size=32, epochs=20, validation_data=(x_test, y_test), verbose=0)
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    model['test_acc'] = test_acc
    model['history'] = history
    print(f'Test Accuracy: {test_acc:.4f}')
    print('-'*50)

In [ ]:

top_models.sort(key=lambda x: x['test_acc'], reverse=True)


for i, model in enumerate(top_models):
    print(f'Top {i+1} Model Test Accuracy: {model["test_acc"]:.4f}')